In [1]:
import os
import numpy as np
import pandas as pd
from joblib import load as load_joblib
from src.models.lstm import LSTMNN
from src.processing_utils.create_timeseries import create_time_series_array
from src.processing_utils.preprocessing import generate_aux_columns, seconds_till_arrival

2023-10-13 21:53:44.508938: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-13 21:53:44.508998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-13 21:53:44.509046: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-13 21:53:44.519610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path_data = os.path.join("..", "data", "processed", "training_data_2022_10sec_sample.csv")
df = pd.read_csv(path_data, parse_dates=["arrival_time", "timestamp"])
df.head()

,flight_id,timestamp,distance,altitude,geoaltitude,arrival_time,vertical_rate,groundspeed,track,latitude,longitude
0,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:37+00:00,1824.713200,4125.0,4450.0,2022-01-02 19:03:14+00:00,1600.0,243.0,302.055358,41.395844,28.695665
1,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:46+00:00,1823.690671,4400.0,4775.0,2022-01-02 19:03:14+00:00,2432.0,250.0,301.396912,41.400674,28.685111
2,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:55+00:00,1822.515934,4775.0,5100.0,2022-01-02 19:03:14+00:00,2944.0,252.0,300.846587,41.406214,28.672975
3,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:04+00:00,1821.289283,5225.0,5575.0,2022-01-02 19:03:14+00:00,2368.0,253.0,300.924912,41.411911,28.660209
4,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:13+00:00,1820.253476,5550.0,5875.0,2022-01-02 19:03:14+00:00,2688.0,254.0,301.002392,41.416763,28.649472


In [3]:
df.head(10)

,flight_id,timestamp,distance,altitude,geoaltitude,arrival_time,vertical_rate,groundspeed,track,latitude,longitude
0,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:37+00:00,1824.713200,4125.0,4450.0,2022-01-02 19:03:14+00:00,1600.0,243.0,302.055358,41.395844,28.695665
1,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:46+00:00,1823.690671,4400.0,4775.0,2022-01-02 19:03:14+00:00,2432.0,250.0,301.396912,41.400674,28.685111
2,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:19:55+00:00,1822.515934,4775.0,5100.0,2022-01-02 19:03:14+00:00,2944.0,252.0,300.846587,41.406214,28.672975
3,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:04+00:00,1821.289283,5225.0,5575.0,2022-01-02 19:03:14+00:00,2368.0,253.0,300.924912,41.411911,28.660209
4,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:13+00:00,1820.253476,5550.0,5875.0,2022-01-02 19:03:14+00:00,2688.0,254.0,301.002392,41.416763,28.649472
5,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:22+00:00,1819.008190,6000.0,6350.0,2022-01-02 19:03:14+00:00,2624.0,255.0,300.886746,41.422600,28.636567
6,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:31+00:00,1817.799039,6400.0,6675.0,2022-01-02 19:03:14+00:00,2432.0,256.0,301.079040,41.428280,28.624048
7,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:40+00:00,1816.680966,6725.0,7050.0,2022-01-02 19:03:14+00:00,2496.0,260.0,300.812381,41.433517,28.612456
8,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:49+00:00,1815.786457,7050.0,7375.0,2022-01-02 19:03:14+00:00,2816.0,260.0,300.812381,41.437683,28.603155
9,DLH1299_2022-01-02 16:18:30+00:00,2022-01-02 16:20:58+00:00,1814.341403,7550.0,7850.0,2022-01-02 19:03:14+00:00,2688.0,261.0,301.189923,41.444433,28.588150


In [4]:
scaler_path = os.path.join("..", "trained_models", "std_scaler_reg_new.bin")
scaler = load_joblib(scaler_path)
drop_columns = ["timestamp", "track", "latitude", "longitude", "arrival_time"]
COLS_TO_SCALE = ["distance", "altitude", "geoaltitude", "vertical_rate", "groundspeed"]
arrival_days = df['arrival_time'].dt.date.unique()
df['flight_id'] = df['flight_id'].astype(str) + '_' + df['arrival_time'].dt.date.astype(str)


flight_ids_train = df['flight_id'].unique()
df = generate_aux_columns(df)
y = seconds_till_arrival(df)/df.distance
df = df.drop(columns=drop_columns)


In [5]:
X_numeric = df[COLS_TO_SCALE]
df[COLS_TO_SCALE] = scaler.transform(X_numeric)
df["time_to_arrival"] = y.values

time_series_array = create_time_series_array(
    df,
    stepsize=1,
    sequence_length=10,
    flight_ids=flight_ids_train[0:10],
    apply_padding=False,
    stride=1,
)

100%|██████████| 10/10 [00:28<00:00,  2.87s/it]                  


In [ ]:
X = time_series_array[:, :, :-1]  # last column is the time_to_arrival
y = time_series_array[:, -1, -1]  # the last y-value in the sequence is the target
model = LSTMNN(scaler=scaler, distance_relative=True, n_features = X.shape[2])
model.fit(X, y, X, y)

Epoch 1/2000
294/294 [==============================] - 15s 26ms/step - loss: 1.8285 - val_loss: 1.0920 - lr: 1.0000e-04
Epoch 2/2000
294/294 [==============================] - 7s 23ms/step - loss: 1.2422 - val_loss: 0.9538 - lr: 1.0000e-04
Epoch 3/2000
294/294 [==============================] - 6s 21ms/step - loss: 1.1073 - val_loss: 0.5859 - lr: 1.0000e-04
Epoch 4/2000
294/294 [==============================] - 6s 19ms/step - loss: 0.9397 - val_loss: 0.7316 - lr: 1.0000e-04
Epoch 5/2000
294/294 [==============================] - 5s 18ms/step - loss: 0.8824 - val_loss: 0.5167 - lr: 1.0000e-04
Epoch 6/2000
294/294 [==============================] - 5s 18ms/step - loss: 0.8201 - val_loss: 0.5763 - lr: 1.0000e-04
Epoch 7/2000
294/294 [==============================] - 6s 19ms/step - loss: 0.7879 - val_loss: 0.5716 - lr: 1.0000e-04
Epoch 8/2000
294/294 [==============================] - 6s 19ms/step - loss: 0.7989 - val_loss: 0.5182 - lr: 1.0000e-04
Epoch 9/2000
149/294 [==============>..

In [81]:
predictor.map

In [64]:
X[:40, 0]

array([[-0.60784809, -1.22600795, -1.23692427, -0.43560267, -1.38644991,
         0.        , -0.7659977 , -0.64284332, -0.49377555,  0.86958939,
        -0.99476328,  0.1022058 ,  0.98659827,  0.16316818,  0.8740863 ,
         0.13733533,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ],
       [-0.60907345, -1.24833523, -1.25705934, -0.3593098 , -1.48609404,
         0.        , -0.76688519, -0.64178431, -0.49377555,  0.86958939,
        -0.99220967,  0.12457918,  0.92901867,  0.37003286,  0.87422199,
         0.13789891,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ],
       [-0.60916765, -1.37417995, -1.37585627,  0.02215453, -0.65904774,
         0.        ,  0.99925171, -0.03867848, -0.49377555,  0.86958939,
         0.94207051, -0.33541489, -0.95774464,  0.28761991,  0.87078656,
         0.16041327,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.    

In [71]:
track_cos

0.2821663239915501

In [72]:
direction_rad = math.atan2(track_sin, track_cos)

In [73]:
math_angle_rad = math.pi/2 - direction_rad
math_angle_rad

2.855541211872475

In [75]:
math.degrees(direction_rad)

-73.61045966596522

In [76]:
direction_rad

-1.2847448850775784

In [80]:
image = Image.open("plane.png")
image = image.resize((30,30))

In [81]:
image.save("plane_small.png")